In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

fullData=[]
for pageNumber in range(1,500):
    print("Getting current page number {}".format(pageNumber))
    initData=requests.get('http://techspy.com/?tab=Home&page={}'.format(pageNumber)).content
    initData=initData.replace('\r','').replace('\n','')
    initsoup = BeautifulSoup(initData, 'html.parser')
    # Getting the initial articles
    articleLinks = initsoup.findAll("div", {"class": "si-content"})
    for curArticle in list(articleLinks):
        innerArticle=list(curArticle.children)[0]['href']
        # Getting the detailed articles
        innerData=requests.get('http://www.techspy.com/' + innerArticle).content
        innerData=innerData.replace('\r','').replace('\n','')
        innersoup = BeautifulSoup(innerData, 'html.parser')
        try:
            title=innersoup.find('h1',{"class":" sd-title"}).text
            article=innersoup.find('p',{"class":" sd-text"}).text
            time=innersoup.find('span',{"class":" sd-time"}).text
            fullData.append([title,article,time])
        except:
            print("Moving on")
    print("After pagenumber {} the length is {}".format(pageNumber,len(fullData)))

In [ ]:
from selenium import webdriver
import os

path_to_phantomJS='/home/ubuntu/anant/SAProject/apps/phantomjs'
browser = webdriver.PhantomJS(executable_path = path_to_phantomJS,service_log_path=os.path.devnull)
browser.get('http://techspy.com/?tab=Home&page=1')
pageContents=browser.page_source
browser.close()
browser.quit()

In [ ]:
fullData=[]
initsoup = BeautifulSoup(pageContents, 'html.parser')
# Getting the initial articles
articleLinks = initsoup.findAll("div", {"class": "si-content"})
for curArticle in list(articleLinks):
    innerArticle=list(curArticle.children)[0]['href']
    # Getting the detailed articles
    browser = webdriver.PhantomJS(executable_path = path_to_phantomJS,service_log_path=os.path.devnull)
    browser.get('http://www.techspy.com/' + innerArticle)
    print('http://www.techspy.com/' + innerArticle)
    innerData=browser.page_source
    browser.close()
    browser.quit()
    innerData=innerData.replace('\r','').replace('\n','')
    innersoup = BeautifulSoup(innerData, 'html.parser')
    try:
        title=innersoup.find('h1',{"class":" sd-title"}).text
        article=innersoup.find('p',{"class":" sd-text"}).text
        time=innersoup.find('span',{"class":" sd-time"}).text
        fullData.append([title,article,time])
    except:
        print("Moving on")
print("After pagenumber {} the length is {}".format(pageNumber,len(fullData)))

In [ ]:
from selenium.webdriver.chrome.options import Options
chromeOptions = Options()
chromeOptions.binary_location='/home/ubuntu/anant/SAProject/apps/chromedriver'
chromeOptions.set_headless(True)
chromeOptions.add_argument('--headless')
chromeOptions.add_argument('--no-sandbox')
chromeOptions.add_argument('--disable-dev-shm-usage')
chromeOptions.add_argument("start-maximized")
chromeOptions.add_argument('--disable-gpu')
chromeOptions.add_argument("disable-infobars")
chromeOptions.add_argument("--disable-extensions")
chromeOptions.experimental_options['useAutomationExtension']=False

In [ ]:
# KNOWTECHIE DATA
#FullData=[]
path_to_phantomJS='/home/ubuntu/anant/SAProject/apps/phantomjs'
path_to_chromeJS='/home/ubuntu/anant/SAProject/apps/chromedriver'
#browser = webdriver.PhantomJS(executable_path = path_to_phantomJS,service_log_path=os.path.devnull)
browser = webdriver.Chrome(chrome_options=chromeOptions,executable_path = path_to_chromeJS)
browser.get('https://knowtechie.com/category/news/')
print('https://knowtechie.com/category/news/')
clickElem=browser.find_element_by_class_name("mvp-inf-more-but")
print(clickElem)
clickElem.click()
clickElem=browser.find_element_by_class_name("mvp-inf-more-but")
print(clickElem)
clickElem.click()
#browser.find_element_by_class_name("mvp-inf-more-but").click()
innerData=browser.page_source
browser.close()
browser.quit()
Data=innerData.replace('\r','').replace('\n','')
Soup = BeautifulSoup(innerData, 'html.parser')
getKnowTechieData(Soup,FullData)

In [ ]:
def getKnowTechieData(Soup,fullData):
    masterElement=Soup.find('ul',{"class" : "mvp-main-blog-wrap left relative infinite-content"})
    for x in masterElement.find_all('li'):
        fullDataText=''
        curText=x.find('h2').text
        curInnerLink=x.find('h2').find('a')['href']
        browser = webdriver.PhantomJS(executable_path = path_to_phantomJS,service_log_path=os.path.devnull)
        browser.get(curInnerLink)
        print("Getting data for {} ".format(curInnerLink))
        innerData=browser.page_source
        browser.close()
        browser.quit()
        innerData=innerData.replace('\r','').replace('\n','')
        innerSoup = BeautifulSoup(innerData, 'html.parser')
        innerTime=innerSoup.find('time',{"class":"post-date updated"}).text
        dataDiv=innerSoup.find('div',{"id":"mvp-content-main"})
        for dataElem in dataDiv.find_all('p'):
            fullDataText= fullDataText +' ' + dataElem.text
        fullData.append([curInnerLink,innerTime,fullDataText]) 

In [ ]:
# Reading the ALL the News Dataset
import pandas as pd
import os
FullData=pd.DataFrame(columns=['id','title','publication','author','date','year','month','url','content'])
def getData(fileName):
    data=[]
    counter=1
    with open('../data/kaggle/all-the-news/{}'.format(fileName),'r') as f1:
        for line in f1:
            if counter > 1:
                curCounter=line[0:line.find(',')]
                line=line[line.find(',')+1:]
                curId=line[0:line.find(',')]
                line=line[line.find(',')+1:]
                line=line.strip()
                if(len(str(line)) > 0):
                    if(line[0]=='"'):
                        line=line[1:]
                        curTitle=line[0:line.find('"')]
                        line=line[line.find('"')+1:]
                        line=line[line.find(',')+1:]
                    else:
                        # Normal Processing
                        curTitle=line[0:line.find(',')]
                        line=line[line.find(',')+1:]
                if(len(str(line)) > 0):
                    if(line[0]=='"'):
                        line=line[1:]
                        curPublication=line[0:line.find('"')]
                        line=line[line.find('"')+1:]
                        line=line[line.find(',')+1:]
                    else:
                        # Normal Processing
                        curPublication=line[0:line.find(',')]
                        line=line[line.find(',')+1:]
                if(len(str(line)) > 0):
                    if(line[0]=='"'):
                        line=line[1:]
                        curAuthor=line[0:line.find('"')]
                        line=line[line.find('"')+1:]
                        line=line[line.find(',')+1:]
                    else:
                        # Normal Processing
                        curAuthor=line[0:line.find(',')]
                        line=line[line.find(',')+1:]
                if(len(str(line)) > 0):
                    if(line[0]=='"'):
                        line=line[1:]
                        curDate=line[0:line.find('"')]
                        line=line[line.find('"')+1:]
                        line=line[line.find(',')+1:]
                    else:
                        # Normal Processing
                        curDate=line[0:line.find(',')]
                        line=line[line.find(',')+1:]
                if(len(str(line)) > 0):
                    if(line[0]=='"'):
                        line=line[1:]
                        curYear=line[0:line.find('"')]
                        line=line[line.find('"')+1:]
                        line=line[line.find(',')+1:]
                    else:
                        # Normal Processing
                        curYear=line[0:line.find(',')]
                        line=line[line.find(',')+1:]
                if(len(str(line)) > 0):
                    if(line[0]=='"'):
                        line=line[1:]
                        curMonth=line[0:line.find('"')]
                        line=line[line.find('"')+1:]
                        line=line[line.find(',')+1:]
                    else:
                        # Normal Processing
                        curMonth=line[0:line.find(',')]
                        line=line[line.find(',')+1:]
                if(len(str(line)) > 0):
                    if(line[0]=='"'):
                        line=line[1:]
                        curURL=line[0:line.find('"')]
                        line=line[line.find('"')+1:]
                        line=line[line.find(',')+1:]
                    else:
                        # Normal Processing
                        curURL=line[0:line.find(',')]
                        line=line[line.find(',')+1:]
                #curPublication=line[0:line.find(',')]
                #line=line[line.find(',')+1:]
                #curAuthor=line[0:line.find(',')]
                #line=line[line.find(',')+1:]
                #curDate=line[0:line.find(',')]
                #line=line[line.find(',')+1:]
                #curYear=line[0:line.find(',')]
                #line=line[line.find(',')+1:]
                #curMonth=line[0:line.find(',')]
                #line=line[line.find(',')+1:]
                #curURL=line[0:line.find(',')]
                line=line[line.find(',')+2:]
                curContent=line
                data.append([curId,curTitle,curPublication,curAuthor,curDate,curYear,curMonth,curURL,curContent])
            counter=counter+1
    data=pd.DataFrame(data,columns=['id','title','publication','author','date','year','month','url','content'])
    return(data)

for curFileName in os.listdir('../data/kaggle/all-the-news/'):
    print(curFileName)
    #FullData=pd.concat([FullData,getData(curFileName)])
    FullData=pd.concat([FullData,pd.read_csv('../data/kaggle/all-the-news/{}'.format(curFileName),sep=',',error_bad_lines=False)])
    print(FullData.shape)
    
#FullData.to_csv('../data/allNewsFullData.txt',sep=chr(255),index=False)

In [2]:
# Read all the news data
# GNF DATA
#!head -5  /home/ubuntu/anant/SAProject/data/kaggle/GNF/file1aa
import pandas as pd
import os
GNFData=pd.DataFrame(columns=['publish_time','feed_code','source_url','headline_text'])
for curFile in os.listdir('../data/kaggle/GNF/'):
    print(curFile)
    curData=pd.read_csv('../data/kaggle/GNF/{}'.format(curFile),skiprows=1,engine='python',sep=',',skip_blank_lines=True,error_bad_lines=False)
    #curData=pd.read_csv('../data/kaggle/GNF/{}'.format(curFile),skiprows=1,error_bad_lines=False)
    curData.columns=['publish_time','feed_code','source_url','headline_text']
    for curcol in curData.columns:
        curData[curcol]=curData[curcol].map(lambda x : ''.join([y for y in str(x) if ord(y) <= 126 and ord(y) >= 32]))
    GNFData=pd.concat([GNFData,curData])
    print(GNFData.shape)
#GNFData.to_csv('../data/GNFHeadLineData.txt',sep=chr(255),index=False)

file1ac.bz2


Error: ',' expected after '"'

In [ ]:
#############################################
# Expansion Algorithm
#############################################
import pandas as pd
import numpy as np
import os
import re

companyData=pd.read_csv('../data/CompanyDataSansLocationFull.txt')
#companyData=pd.read_csv('/home/anantgupta/Documents/Programming/Projects/situationalAwareness/SAProject/CompanyDataSansLocationFull.txt')

products=companyData[companyData['propertyLabel']=='products']['valueLabel'].unique()
companies=companyData['CompanyLabel'].unique()

def processCompany(x):
    # We need to first convert all the non alphanumeric to ''
    x=str(x).replace('"','')
    x=re.sub(r'\([^)]*\)', '',x)
    if(len(x.strip()) <= 3):
        x=''
    return(x.strip())

companyData['companyLabelProcessed']=companyData['CompanyLabel'].map(lambda x : processCompany(x))
companyData.to_csv('../data/CompanyDataSansLocationFull.txt',index=False)

In [ ]:
import nltk
import numpy as np
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))

def getContinousBigrams(inpArray):
    outArray=[]
    for curElement in inpArray:
        tempArray=curElement.split(' ')
        if(len(tempArray) > 2):
            tempArray
        else:
            outArray.append(curElement)

def getDetailedProducts(x,allElements):
    indItems=x.split(',')
    if(len(indItems) > 1):
        indItems=[ ' '.join([str(y).strip().lower() for y in curItem.split(' ') if str(y).lower() not in stopWords]).strip() for curItem in indItems]
        # Now for all the individual items, we will prepare a list of words that we need to search based on importance
        bigrams = [' '.join(b) for l in indItems for b in zip(l.split(" ")[:-1], l.split(" ")[1:])]
        allElements[x]=bigrams
        #allElements.extend(indItems)
    else:
        indItems=' '.join([str(y).strip().lower() for y in indItems[0].split(' ') if str(y).lower() not in stopWords])
        bigrams = [' '.join(b) for l in [indItems] for b in zip(l.split(" ")[:-1], l.split(" ")[1:])]
        allElements[x]=bigrams
        #allElements.append(indItems.strip())
        
allElements={}
[getDetailedProducts(x,allElements) for x in products]
#allElements=list(set(allElements))

In [ ]:
# Some papers
#https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=1&cad=rja&uact=8&ved=2ahUKEwiz7vmB0abeAhXLWysKHVepAB8QFjAAegQICRAB&url=https%3A%2F%2Fstackoverflow.com%2Fquestions%2F21844546%2Fforming-bigrams-of-words-in-list-of-sentences-with-python&usg=AOvVaw1cF0_ZkiXJy-HXku5ozxhd
#Discovering expansion entities for keyword-based entity search in linked data

# Our approach
# We will find the most commoinly occurring grammar linkages between the entity and aspect

def cleanNewsText(x):
    if(x[0:4]=='http'):
        return ''
    else:
        return(''.join([y for y in str(x) if ord(y) <= 126 and ord(y) >= 32]))

FullData['contentNew']=FullData['content'].map(lambda x : cleanNewsText(x))

# Saving the DataFrame
FullData.reset_index(inplace=True)
FullData['contentNew']=FullData[['contentNew']].fillna('').values
FullData=FullData[FullData['contentNew'] != '']
FullData.to_csv('../../SADataCompany.txt',index=False)

# Also saving the company Dataframe
pd.DataFrame(companies,columns=['companyName']).to_csv('../../companyNameList.txt',index=False)

In [10]:
import pandas as pd
#FullData=pd.read_csv('../../SADataCompany.txt')
companyData=pd.read_csv('../data/CompanyDataSansLocationFull.txt')

In [11]:
#products=companyData[companyData['propertyLabel']=='products']['valueLabel'].unique()
products=[]
def getProducts(x,products):
    products.extend([y.strip().lower() for y in x.split(',')])

[getProducts(x,products) for x in companyData[companyData['propertyLabel']=='products']['valueLabel'].unique()]
products=list(set(products))

import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

def convertLemma(x):
    return(' '.join([wordnet_lemmatizer.lemmatize(y) for y in x.split(' ')]))

products=[convertLemma(x) for x in products]

In [17]:
# UCI NEWS
#FullData=pd.read_csv('/home/anantgupta/Documents/Programming/Projects/situationalAwareness/SAProject/data/kaggle/uci-news-aggregator.csv')
#FullData['indexVal']=FullData.reset_index().index.values
FullData.shape

(422419, 9)

In [18]:
#FullData.head(5)
#np.range(FullData.shape[0],1000)
#np.arange(1,FullData.shape[0],1000)
#df1, df2 = np.split(df, [72], axis=1)

import time
from threading import Thread
from multiprocessing.dummy import Pool as ThreadPool

companies=companyData['companyLabelProcessed'].values
companies=[str(x) for x in companies]

# This method is very naive, as it does not make the split checks, but for now we will go ahead with this only
# This clearly requires refinement
results=[]
def findCompanies(x):
    if len(x) > 10:
        matches=list(set([y for y in companies if y in x]))
        return([str(curMatch) for curMatch in matches if len(str(curMatch)) > 1])
    else:
        return([''])

def getNewsBatch(batchIndex,curList,results):
    print "Started Processing for the batch start index {}".format(batchIndex)
    results.append([[x[0],findCompanies(str(x[1]))] for x in curList])
    print "Finished Processing for the batch start index {}".format(batchIndex)

def calculateParallel(numbers, threads=2):
    pool = ThreadPool(100)
    results = pool.map(getNewsBatch, numbers)
    pool.close()
    pool.join()
    return results
    
threadList=[]
#for i in range(0,143000,100):
#for i in range(14300,143000,100):
for i in range(0,10000,100):
    batchIndex=i
    curList=FullData[i:i+100][['indexVal','TITLE']].values
    threadList.append(Thread(target=getNewsBatch, args=(batchIndex,curList,results,)))

#for curIter in range(0,1430-143,20):
for curIter in range(0,100,20):
    startTime=time.time()
    curThreadList=threadList[curIter:curIter + 20]
    for curThread in curThreadList:
        curThread.start()
    for curThread in curThreadList:
        curThread.join()
    print("Completed for the following batch of 20 threads {} in {} seconds".format(curIter,str(time.time() - startTime)))
    
resultsPD=[]
for x in results:
    for y in x:
        resultsPD.append(y)
resultsPD=pd.DataFrame(resultsPD,columns=['index','companyMatch'])
resultsPD['lengthVal']=resultsPD['companyMatch'].map(lambda x : len(x))

Started Processing for the batch start index 0 
Finished Processing for the batch start index 0Started Processing for the batch start index 100Started Processing for the batch start index 400Started Processing for the batch start index 300Started Processing for the batch start index 500Started Processing for the batch start index 600Started Processing for the batch start index 700 


Started Processing for the batch start index 200



Started Processing for the batch start index 800Started Processing for the batch start index 900 Started Processing for the batch start index 1000Started Processing for the batch start index 1300Started Processing for the batch start index 1200   Started Processing for the batch start index 1700Started Processing for the batch start index 1800
Finished Processing for the batch start index 100Finished Processing for the batch start index 400Started Processing for the batch start index 1900
Finished Processing for the batch start index 300Finished Processin

Started Processing for the batch start index 8600Started Processing for the batch start index 8700 Started Processing for the batch start index 9000    Started Processing for the batch start index 9400Started Processing for the batch start index 9600 

Started Processing for the batch start index 9700Started Processing for the batch start index 9500Finished Processing for the batch start index 7500

Finished Processing for the batch start index 8500
Started Processing for the batch start index 9800




Started Processing for the batch start index 9300Finished Processing for the batch start index 8000
Started Processing for the batch start index 9200

Started Processing for the batch start index 8800
Started Processing for the batch start index 9100Started Processing for the batch start index 8900



Finished Processing for the batch start index 8400 
Finished Processing for the batch start index 9000
Finished Processing for the batch start index 9400Finished Processing for the batch st

In [ ]:
#resultsPD.to_csv('../../companyMatches.txt',index=False)
#FullData[['index','contentNew']].merge(resultsPD[['index','companyMatch']],left_on='index',right_on='index',how='inner').head(5)
#resultsPD[resultsPD['lengthVal'] > 0]
#FullData[['index','contentNew']].merge(resultsPD[['index','companyMatch']],left_on='index',right_on='index',how='inner').head(5)
#FullData[FullData['index']==1]
FullData.head(5)

In [ ]:
#FullData['index']=FullData['index'].values.astype(int)
#FullData['index'].values
#FullData[['index','contentNew']].merge(resultsPD[['index','companyMatch']],left_on='index',right_on='index',how='inner').head(5)

In [ ]:
resultsPD=resultsPD.drop_duplicates()
FullData['index']=FullData['index'].values.astype(int)

# Merging the data
workingData=FullData.merge(resultsPD,left_on='index',right_on='index',how='inner')

# Writing the data
workingData.to_csv('../../workingData.csv',index=False)

In [ ]:
import time
from threading import Thread
from multiprocessing.dummy import Pool as ThreadPool

# PRODUCTS MATCHING
products=[str(x) for x in products if len(str(x)) > 3]

# This method is very naive, as it does not make the split checks, but for now we will go ahead with this only
# This clearly requires refinement
results=[]
def findProducts(x):
    if len(x) > 10:
        matches=list(set([y for y in products if y in x]))
        return([str(curMatch) for curMatch in matches if len(str(curMatch)) > 1])
    else:
        return([''])

def getNewsBatch(batchIndex,curList,results):
    print "Started Processing for the batch start index {}".format(batchIndex)
    results.append([[x[0],findProducts(str(x[1]))] for x in curList])
    print "Finished Processing for the batch start index {}".format(batchIndex)

def calculateParallel(numbers, threads=2):
    pool = ThreadPool(100)
    results = pool.map(getNewsBatch, numbers)
    pool.close()
    pool.join()
    return results
    
threadList=[]
#for i in range(0,143000,100):
#for i in range(14300,143000,100):
for i in range(0,14300,100):
    batchIndex=i
    curList=FullData[i:i+100][['index','contentNew']].values
    threadList.append(Thread(target=getNewsBatch, args=(batchIndex,curList,results,)))

#for curIter in range(0,1430-143,20):
for curIter in range(0,1430,20):
    startTime=time.time()
    curThreadList=threadList[curIter:curIter + 20]
    for curThread in curThreadList:
        curThread.start()
    for curThread in curThreadList:
        curThread.join()
    print("Completed for the following batch of 50 threads {} in {} seconds".format(curIter,str(time.time() - startTime)))

resultsPD1=[]
for x in results:
    for y in x:
        resultsPD1.append(y)
resultsPD1=pd.DataFrame(resultsPD1,columns=['index','productMatch'])
resultsPD1=resultsPD1.drop_duplicates()
resultsPD1.to_csv('../../productMatches.txt',index=False)
# Read back the data with the company Filter
#workingData=pd.read_csv('../../workingData.csv')

# Merging the data with the product match
#workingData=workingData.merge(resultsPD,left_on='index',right_on='index',how='inner')

# Writing the data back 
#workingData.to_csv('../../workingDataWithProducts.csv',index=False)

In [ ]:
resultsPD1.to_csv('../../productMatches.txt',index=False)

In [ ]:
resultsPD=[]
for x in results:
    for y in x:
        resultsPD.append(y)
resultsPD=pd.DataFrame(resultsPD,columns=['index','productMatch'])
workingData=pd.read_csv('../../workingData.csv')

# Merging the data
workingData=workingData.merge(resultsPD,left_on='index',right_on='index',how='inner')

# Writing the data
workingData.to_csv('../../workingDataWithProducts.csv',index=False)

In [ ]:
import nltk
import nltk, re, pprint
from nltk import word_tokenize
import requests
import urllib
import sys

def getSubjects(phrase):
    phrase="\""+phrase+"\""
    query="""PREFIX dbr: <http://dbpedia.org/resource/>
    PREFIX bif:<>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX rdf:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX dbpedia-owl: <http://dbpedia.org/ontology/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/> 
    SELECT DISTINCT * WHERE 
    {
     ?s rdfs:label ?label .
     ?sub ?p ?s .
     FILTER (CONTAINS(lcase(?label),"""+phrase+"""))
    }"""
    #print(query)
    q=urllib.parse.quote_plus(query)
    #print(queryLocalDB(q))
    return queryLocalDB(q)

def queryLocalDB(q):
    predicateList=[]
    with open(r'C:\Users\Administrator\Scripts_Technophilia\predicateList1.txt') as f:
        content = f.readlines()
    for x in content:
        x=x[1:-2]
        #print(x)
        predicateList.append(x)
    #predicateList = [x.strip() for x in content] 
    #print(predicateList)
    url = "http://172.16.131.14:3030/da/query"
    #print(q)
    
    payload = "query="+q
    headers = {
        'Content-Type': "application/x-www-form-urlencoded",
        'cache-control': "no-cache",
        'Postman-Token': "681d8312-7fab-40ab-b91d-e1926ad703cd"
        }

    response = requests.request("POST", url, data=payload, headers=headers)
#     print(response.text)
#     print(response.json().keys())
#     print(response.json()['results']['bindings'])
#     print('I am in query function')
#     print(len(response.json()['results']['bindings']))
    subjects=[]
    superSubjects=[]
    for i in response.json()['results']['bindings']:
        #print('here we go')
        #print(i['s']['value'])
        #print(i['sub']['value'])
        #print(i['p']['value'])
        if i['p']['value'] in predicateList:
            #print('hello, hi')
            #print(i['p']['value'])
            subjects.append(i['s']['value'])
            superSubjects.append(i['sub']['value'])
    #print('subjects')
    #print(subjects)
    return subjects,superSubjects

#to preprocess query :
# stop words removal
# NER
#Nouns phrases
#nouns
#verbs

        
    
        
    
	
def parseChunking(parser, postags):
    chunked = parser.parse(postags)
#     print(type(chunked))
#     print(type(postags))
    for subtree in chunked.subtrees():
        print(subtree)
    phrases=[]
    for subtree in chunked.subtrees(filter=lambda t: t.label() == 'Chunk'):
        np=''
#         print(subtree)
        for s in subtree:
#             print(s[0], s[1])
            np=np+s[0]+' '
        phrases.append(np.rstrip())    
    return phrases
    

def queryPreprocessing(query):
    phraseList=[]
    tokens = word_tokenize(query)
    postags=nltk.pos_tag(tokens)
#     print(postags)
    #chunkGram = r"""Chunk: {<RB.?>*<VB.?>*<NNP>+<NN>?}"""
    chunkGram = r"""Chunk: {<RB.?>*<VB.?>*<NN>+<NNS>?}"""
    chunkGram2 = r"""Chunk: {<RB.?>*<VB.?>*<NNP>+<NN>?}"""
    chunkParser = nltk.RegexpParser(chunkGram)
    chunkParser2 = nltk.RegexpParser(chunkGram2)
    #chunked = chunkParser.parse(postags)
    phraseList.extend(parseChunking(chunkParser,postags))
    phraseList.extend(parseChunking(chunkParser2,postags))
    
    #print(phraseList)
    return phraseList
        
    
def main():
    phrases =queryPreprocessing('services industry in america')
    subjects=[]
    print(phrases)
    for phrase in phrases:
        if len(phrase)>3:
            print(phrase)
            subjects, superSubjects=getSubjects(phrase.lower())
        #subjects.append(getSubjects(phrase.lower()))
    print(len(set(subjects)))
    print(len(set(superSubjects)))
   
if __name__ == "__main__":
    main()